In [5]:
# Temporary import. It will be removed in the final vserion
import os
import sys

import os
os.chdir('/Users/zhanwenxin/Documents/GitHub/cuTAGI-LSTM')

from typing import Union, Optional

import fire
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from tqdm import tqdm

import pytagi.metric as metric
from pytagi import Normalizer as normalizer
from pytagi import exponential_scheduler
from pytagi.nn import LSTM, Linear, OutputUpdater, Sequential

from examples.data_loader import TimeSeriesDataloader
from pytagi.hybrid import LSTM_SSM
from pytagi.hybrid import process_input_ssm

In [6]:
num_epochs: int = 50
batch_size: int = 1
sigma_v: float = 1E-12

"""Run training for time-series forecasting model"""
# Dataset
output_col = [0]
num_features = 2
input_seq_len = 26
output_seq_len = 1
seq_stride = 1

train_dtl = TimeSeriesDataloader(
    x_file="data/HQ/hq_ts5_train.csv",
    date_time_file="data/HQ/hq_ts5_train_datetime.csv",
    output_col=output_col,
    input_seq_len=input_seq_len,
    output_seq_len=output_seq_len,
    num_features=num_features,
    stride=seq_stride,
    time_covariates = ['week_of_year'],  # 'hour_of_day','day_of_week', 'week_of_year', 'month_of_year','quarter_of_year'
)
test_dtl = TimeSeriesDataloader(
    x_file="data/HQ/hq_ts5_test.csv",
    date_time_file="data/HQ/hq_ts5_test_datetime.csv",
    output_col=output_col,
    input_seq_len=input_seq_len,
    output_seq_len=output_seq_len,
    num_features=num_features,
    stride=seq_stride,
    x_mean=train_dtl.x_mean,
    x_std=train_dtl.x_std,
    time_covariates = ['week_of_year'],  # 'hour_of_day','day_of_week', 'week_of_year'
)

#### Use a first-order regressor to estimate the baseline initial values

In [7]:
x_data = np.array(pd.read_csv("data/HQ/hq_ts5_train.csv", skiprows=1, delimiter=",", header=None).values.T[0])
x_data = normalizer.standardize(data=x_data, mu=train_dtl.x_mean[output_col], std=train_dtl.x_std[output_col])
time_idx = np.arange(0, len(x_data))

# Fit a first-order regression model, ax+b, to the data x_data, y_data
valid_indices = ~np.isnan(x_data)
time_idx_filtered = time_idx[valid_indices]
x_data_filtered = x_data[valid_indices]
speed_init, level_init = np.polyfit(time_idx_filtered, x_data_filtered, 1)

print(f"Initial level: {level_init}, Initial speed: {speed_init}")

Initial level: -0.6981364030891123, Initial speed: 0.009008211652762738


In [8]:
# Network
net = Sequential(
    LSTM(num_features, 30, input_seq_len),
    LSTM(30, 30, input_seq_len),
    Linear(30 * input_seq_len, 1),
)
net.set_threads(8)
# #net.to_device("cuda")

# # # State-space models: for baseline hidden states
phi_AA = 0.999
Sigma_AR = 0.1**2
Sigma_AA = Sigma_AR*1e-17
LA_var_stationary = Sigma_AA/(1-phi_AA**2)
# # Autoregressive acceleration + online AR
hybrid = LSTM_SSM(
    neural_network = net,           # LSTM
    baseline = 'AA + AR', # 'level', 'trend', 'acceleration', 'ETS'
    zB  = np.array([level_init, speed_init, 0, 0.5, 0.05]),    # initial mu for baseline hidden states
    SzB = np.array([1E-5, 1E-8, LA_var_stationary, 0.25, Sigma_AR]),    # var
    phi_AA = phi_AA,
    Sigma_AR = Sigma_AR,
    Sigma_AA = Sigma_AA,
    use_online_AR = True,
)
# hybrid = LSTM_SSM(
#     neural_network = net,           # LSTM
#     baseline = 'AA + plain_AR', # 'level', 'trend', 'acceleration', 'ETS'
#     zB  = np.array([0.1, 1E-18, 0, 0.02]),    # initial mu for baseline hidden states
#     SzB = np.array([1E-5, 1E-5, LA_var_stationary, 1E-5])    # var
# )
# # Online AR
# hybrid = LSTM_SSM(
#     neural_network = net,           # LSTM
#     baseline = 'trend + AR', # 'level', 'trend', 'acceleration', 'ETS'
#     zB  = np.array([0.1, 1E-4, 0.5, 0.02]),      # initial mu for baseline hidden states
#     SzB = np.array([1E-5, 1E-8, 0.25, Sigma_AR])    # var
# )
# hybrid = LSTM_SSM(
#     neural_network = net,           # LSTM
#     baseline = 'LT + plain_AR', # 'level', 'trend', 'acceleration', 'ETS'
#     zB  = np.array([level_init, speed_init, 0.05]),    # initial mu for baseline hidden states
#     SzB = np.array([1E-5, 1E-8, 1E-1]),    # var
#     # Sigma_AR = 0.1**2,
#     Sigma_AR = Sigma_AR,
#     phi_AR = 0.91225,
#     use_online_AR = False,
# )

# -------------------------------------------------------------------------#
# Training
mses = []

pbar = tqdm(range(num_epochs), desc="Training Progress")
for epoch in pbar:
    batch_iter = train_dtl.create_data_loader(batch_size, shuffle=False)

    # Decaying observation's variance
    sigma_v = exponential_scheduler(
        curr_v=1E-12, min_v=1E-12, decaying_factor=1, curr_iter=epoch
    )
    var_y = np.full((batch_size * len(output_col),), sigma_v**2, dtype=np.float32)

    # Initialize list to save
    hybrid.init_ssm_hs()
    mu_preds_lstm = []
    var_preds_lstm = []
    mu_preds_unnorm = []
    obs_unnorm = []
    mu_phiar = []
    var_phiar = []
    mu_aa = []
    var_aa = []

    for x, y in batch_iter:
        mu_x, var_x = process_input_ssm(
            mu_x = x, mu_preds_lstm = mu_preds_lstm, var_preds_lstm = var_preds_lstm,
            input_seq_len = input_seq_len, num_features = num_features,
            )

        # Feed forward
        y_pred, _, z_pred, Sz_pred, m_pred, v_pred = hybrid(mu_x, var_x)
        # Backward
        hybrid.backward(mu_obs = y, var_obs = var_y)

        # Training metric
        pred = normalizer.unstandardize(
            y_pred.flatten(), train_dtl.x_mean[output_col], train_dtl.x_std[output_col]
        )
        obs = normalizer.unstandardize(
            y, train_dtl.x_mean[output_col], train_dtl.x_std[output_col]
        )
        mse = metric.mse(pred, obs)
        mses.append(mse)
        mu_preds_lstm.extend(m_pred)
        var_preds_lstm.extend(v_pred)
        obs_unnorm.extend(y)
        mu_preds_unnorm.extend(y_pred)
        mu_phiar.append(z_pred[-3].item())
        var_phiar.append(Sz_pred[-3][-3])
        mu_aa.append(z_pred[2].item())
        var_aa.append(Sz_pred[2][2])

    # Smoother
    hybrid.smoother()

    mu_smoothed = np.array(hybrid.mu_smoothed)
    cov_smoothed = np.array(hybrid.cov_smoothed)

    # Figures for each epoch
    plt.switch_backend('Agg')
    plt.figure()
    plt.plot(obs_unnorm, color='r',label=r"obs.")
    plt.plot(mu_smoothed[:,0,:],color='k',label=r"level")
    plt.fill_between(np.arange(len(mu_smoothed[:,0,:])), np.array(mu_smoothed[:,0,:]).flatten() - np.sqrt(cov_smoothed[:, 0, 0]), np.array(mu_smoothed[:,0,:]).flatten() + np.sqrt(cov_smoothed[:, 0, 0]), color='k', alpha=0.3, label='_nolegend_')
    plt.plot(mu_smoothed[:,-2,:],color='g',label=r"AR")
    plt.fill_between(np.arange(len(mu_smoothed[:,-2,:])), np.array(mu_smoothed[:,-2,:]).flatten() - np.sqrt(cov_smoothed[:, -2, -2]), np.array(mu_smoothed[:,-2,:]).flatten() + np.sqrt(cov_smoothed[:, -2, -2]), color='g', alpha=0.3, label='_nolegend_')
    plt.plot(mu_preds_unnorm,color='b',label=r"pred.")
    plt.plot(mu_smoothed[:,-1,:],color='orange',label=r"LSTM")
    plt.fill_between(np.arange(len(mu_smoothed[:,-1,:])), np.array(mu_smoothed[:,-1,:]).flatten() - np.sqrt(cov_smoothed[:, -1, -1]), np.array(mu_smoothed[:,-1,:]).flatten() + np.sqrt(cov_smoothed[:, -1, -1]), color='orange', alpha=0.3, label='_nolegend_')
    plt.legend(ncol = 3, loc='upper left')
    filename = f'saved_results/hq_ts5/hybrid_epoch_#{epoch}.png'
    plt.savefig(filename)
    plt.close()  # Close the plot to free up memory

    # figure for AR
    plt.figure()
    plt.plot(np.arange(len(mu_phiar)),mu_phiar,color='b',label=r"AR")
    plt.fill_between(np.arange(len(mu_phiar)), np.array(mu_phiar) - np.sqrt(var_phiar), np.array(mu_phiar) + np.sqrt(var_phiar), color='blue', alpha=0.3, label='±1 SD')
    plt.ylim(-0.1, 1.1)
    filename = f'saved_results/hq_ts5/phi_AR_epoch_#{epoch}.png'
    plt.savefig(filename)
    plt.close()  # Close the plot to free up memory

    # figure for AA
    plt.figure()
    plt.plot(np.arange(len(mu_aa)),mu_aa,color='b',label=r"AR")
    plt.fill_between(np.arange(len(mu_aa)), np.array(mu_aa) - np.sqrt(var_aa), np.array(mu_aa) + np.sqrt(var_aa), color='blue', alpha=0.3, label='±1 SD')
    filename = f'saved_results/hq_ts5/AA_epoch_#{epoch}.png'
    plt.savefig(filename)
    plt.close()  # Close the plot to free up memory

    # Progress bar
    pbar.set_description(
        f"Epoch {epoch + 1}/{num_epochs}| mse: {np.nanmean(mses):>7.2f}",
        refresh=True,
    )

# print the value of AR
print(f"phi_AR: {mu_phiar[-1]}")

# -------------------------------------------------------------------------#
# Testing
test_batch_iter = test_dtl.create_data_loader(batch_size, shuffle=False)

# Initialize list to save
mu_preds = []
var_preds = []
y_test = []
obs_test_unnorm = []
#

for x, y in test_batch_iter:
    mu_x, var_x = process_input_ssm(
        mu_x = x, mu_preds_lstm = mu_preds_lstm, var_preds_lstm = var_preds_lstm,
        input_seq_len = input_seq_len,num_features = num_features,
    )
    # Feed forward
    y_pred, Sy_red, z_pred, Sz_pred, m_pred, v_pred = hybrid(mu_x, var_x)
    hybrid.backward(mu_obs = np.nan, var_obs = np.nan, train_LSTM=False)

    mu_preds.extend(y_pred)
    var_preds.extend(Sy_red + sigma_v**2)
    y_test.extend(y)
    mu_preds_lstm.extend(m_pred)
    var_preds_lstm.extend(v_pred)

mu_preds = np.array(mu_preds)
std_preds = np.array(var_preds) ** 0.5
y_test = np.array(y_test)
obs_test_unnorm = y_test
mu_preds_unnorm_test = mu_preds
std_preds_unnorm_test = std_preds

mu_preds = normalizer.unstandardize(
    mu_preds, train_dtl.x_mean[output_col], train_dtl.x_std[output_col]
)
std_preds = normalizer.unstandardize_std(std_preds, train_dtl.x_std[output_col])

y_test = normalizer.unstandardize(
    y_test, train_dtl.x_mean[output_col], train_dtl.x_std[output_col]
)

# Compute log-likelihood
mse = metric.mse(mu_preds, y_test)
log_lik = metric.log_likelihood(
    prediction=mu_preds, observation=y_test, std=std_preds
)

#
obs = np.concatenate((obs_unnorm,obs_test_unnorm), axis=0)
idx_train = range(0,len(obs_unnorm))
idx_test = range(len(obs_unnorm),len(obs))
idx = np.concatenate((idx_train,idx_test),axis=0)
mu_preds_unnorm_test = mu_preds_unnorm_test.flatten()
std_preds_unnorm_test = std_preds_unnorm_test.flatten()

# figure for final test predictions
plt.figure()
plt.plot(idx,obs, color='r',label=r"data")
plt.plot(idx_test, mu_preds_unnorm_test, color='b',label=r"test prediction")
plt.fill_between(idx_test, mu_preds_unnorm_test - std_preds_unnorm_test, mu_preds_unnorm_test + std_preds_unnorm_test, color='blue', alpha=0.3, label='±1 SD')
plt.plot(idx_train,mu_smoothed[:,0,:],color='k',label=r"level")
plt.plot(idx_train, mu_preds_unnorm,color='g', label=r"train prediction")
filename = f'saved_results/hq_ts5/test_prediction1.png'
plt.savefig(filename)
plt.close()  # Close the plot to free up memory

print("#############")
print(f"MSE           : {mse: 0.2f}")
print(f"Log-likelihood: {log_lik: 0.2f}")

Training Progress:   0%|          | 0/50 [00:00<?, ?it/s]

[-2.306773   -0.4330928  -2.306773   -0.36646312 -1.8113561  -0.29983348
 -1.6698085  -0.23320381 -1.7405823  -0.16657415 -1.0328441  -0.09994449
 -1.5282608  -0.03331483 -0.9620703   0.03331483 -1.2805525   0.09994449
 -0.8276      0.16657415 -0.15524873  0.23320381 -0.438344    0.29983348
 -0.01370109  0.36646312  0.2693942   0.4330928   0.09953703  0.49972245
  0.51710254  0.5663521   0.8780491   0.6329818   0.6515728   0.6996114
  0.99836457  0.7662411   1.1469896   0.8328708   0.6515728   0.8995004
  1.0125194   0.9661301   0.6657276   1.0327598   0.6657276   1.0993894
  0.24816205  1.1660191   0.38970968  1.2326487 ]
[0.01665916]
[-2.306773   -0.36646312 -1.8113561  -0.29983348 -1.6698085  -0.23320381
 -1.7405823  -0.16657415 -1.0328441  -0.09994449 -1.5282608  -0.03331483
 -0.9620703   0.03331483 -1.2805525   0.09994449 -0.8276      0.16657415
 -0.15524873  0.23320381 -0.438344    0.29983348 -0.01370109  0.36646312
  0.2693942   0.4330928   0.09953703  0.49972245  0.51710254  0.

Training Progress:   0%|          | 0/50 [00:01<?, ?it/s]


ZeroDivisionError: division by zero

In [ ]:
print(mu_phiar[-1])
print(mu_smoothed[:,0,:])

0.86182671754275
[[-2.45233611e-01]
 [-2.36122732e-01]
 [-2.27022714e-01]
 [-2.17957325e-01]
 [-2.08882980e-01]
 [-1.99803560e-01]
 [-1.90735413e-01]
 [-1.81659620e-01]
 [-1.72583056e-01]
 [-1.63504534e-01]
 [-1.54419353e-01]
 [-1.45342140e-01]
 [-1.36273671e-01]
 [-1.27213073e-01]
 [-1.18135878e-01]
 [-1.09060722e-01]
 [-9.99836540e-02]
 [-9.09033763e-02]
 [-8.18259284e-02]
 [-7.27452155e-02]
 [-6.36677533e-02]
 [-5.45891198e-02]
 [-4.55125056e-02]
 [-3.64327874e-02]
 [-2.73522138e-02]
 [-1.82763809e-02]
 [-9.19648039e-03]
 [-1.14788401e-04]
 [ 8.96430194e-03]
 [ 1.80438541e-02]
 [ 2.71239608e-02]
 [ 3.62032935e-02]
 [ 4.52909260e-02]
 [ 5.43679673e-02]
 [ 6.34546039e-02]
 [ 7.25355697e-02]
 [ 8.16058598e-02]
 [ 9.07082003e-02]
 [ 9.98055809e-02]
 [ 1.08871006e-01]
 [ 1.17923622e-01]
 [ 1.26999851e-01]
 [ 1.36064900e-01]
 [ 1.45174594e-01]
 [ 1.54300665e-01]
 [ 1.63343164e-01]
 [ 1.72419879e-01]
 [ 1.81464259e-01]
 [ 1.90567752e-01]
 [ 1.99623459e-01]
 [ 2.08748881e-01]
 [ 2.17824637e

In [ ]:
hybrid.net.save(filename = './saved_param/HQ_hybrid_LSTM/lstm_hqts5_hybrid.pth')